In [4]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from decimal import Decimal

training_file = "YandexRelPredChallenge.txt"

In [ ]:
# Read data into dataframe
columns = ["SessionID", "TimePassed", "TypeOfAction", "TargetID", "RegionID", 1,2,3,4,5,6,7,8,9,10]
df = pd.read_csv(training_file, sep='\t', header=None, names=columns)

gamma = defaultdict(Decimal)
alpha = defaultdict(lambda: defaultdict(Decimal))

print("Training with EM...")

for i in range(100):

  # Initialise sums and counts
  gamma_count = defaultdict(Decimal)
  alpha_count = defaultdict(lambda: defaultdict(Decimal))

  gamma_sum = defaultdict(Decimal)
  alpha_sum = defaultdict(lambda: defaultdict(Decimal))

  # Iterate sessions
  grouped = df.groupby("SessionID")
  for session_id, session_df in grouped:

    # Extract session clicks
    session_clicks = session_df[session_df["TypeOfAction"] == "C"]["TargetID"].tolist()

    # Iterate session queries
    for index, row in session_df[session_df["TypeOfAction"] == "Q"].iterrows():
      query_id = row["TargetID"]
      for rank in range(1, 11): # from rank 1 to 10
        document_id = row[rank]
        
        # Determine what values should be added to the EM formula sums 
        if document_id in session_clicks:
          gamma_value = alpha_value = 1
        else:
          gamma_value = (gamma[rank] * (1 - alpha[document_id][query_id])) / \
                        (1 - gamma[rank] * alpha[document_id][query_id])
          alpha_value = ((1 - gamma[rank]) * alpha[document_id][query_id]) / \
                        (1 - gamma[rank] * alpha[document_id][query_id])

        gamma_sum[rank] += gamma_value
        alpha_sum[document_id][query_id] += alpha_value

        gamma_count[rank] += 1
        alpha_count[document_id][query_id] += 1    

  # Update variables
  for rank, param in gamma.items():
    gamma[rank] = gamma_sum[rank] / gamma_count[rank]
    
  for document_id, document_params in alpha.items():
    for query_id, param in document_params.items():
      alpha[document_id][query_id] = alpha_sum[document_id][query_id] / alpha_count[document_id][query_id]

  print("Completed EM iteration", i)
  print(gamma, "\n")

Training with EM...
Completed EM iteration 0
defaultdict(<class 'decimal.Decimal'>, {1: Decimal('0.5210775579105317452874425584'), 2: Decimal('0.2493904154553127637625433743'), 3: Decimal('0.1836021757479133452124167683'), 4: Decimal('0.1440495170214761324205195536'), 5: Decimal('0.1124917940542061333583419300'), 6: Decimal('0.09561099127825189909031229485'), 7: Decimal('0.08449779611741536153052611835'), 8: Decimal('0.07678420707118071837194035450'), 9: Decimal('0.07054768826784207071180718372'), 10: Decimal('0.07021945043608740504548438526')}) 

Completed EM iteration 1
defaultdict(<class 'decimal.Decimal'>, {1: Decimal('0.7488095584515335912277215083'), 2: Decimal('0.4256437490848015154390693642'), 3: Decimal('0.3269981974115213058647808846'), 4: Decimal('0.2629782275265703256985426261'), 5: Decimal('0.2096093758972762393322775675'), 6: Decimal('0.1800013114634592254032834189'), 7: Decimal('0.1602519805670651312377511268'), 8: Decimal('0.1462892869451669851235520006'), 9: Decimal('0

In [ ]:
model.click_prob(0.1, 1, 0)